# 🧠 ADK Artifacts Deep Dive

An in-depth walkthrough of how Artifacts work in Google's ADK framework — from structure to best practices.

## 📚 Table of Contents
1. [What Are Artifacts?](#what-are-artifacts)
2. [Why Use Artifacts](#why-use-artifacts)
3. [Core Concepts](#core-concepts)
    - ArtifactService
    - Artifact Data
    - Filename
    - Versioning
    - Namespacing
4. [Interacting with Artifacts](#interacting-with-artifacts)
    - Configuration
    - Saving, Loading, Listing
5. [ArtifactService Implementations](#artifactservice-implementations)
6. [Best Practices](#best-practices)


## 🧩 What Are Artifacts? <a id="what-are-artifacts"></a>

Artifacts in ADK are versioned, file-like binary blobs that can be tied to a user or session. Unlike `session.state` (meant for small key-value data), artifacts can store **images, PDFs, audio, etc.**

They’re stored using an `ArtifactService` (like GCS or in-memory) and accessed with methods on `ToolContext` or `CallbackContext`.

| Component | Description |
|----------|-------------|
| `data`   | Raw binary (e.g., bytes of a file) |
| `mime_type` | Format specifier (e.g., `"image/png"`, `"application/pdf"`) |
| `filename` | Unique identifier within scope |
| `version` | Automatically handled per filename |

Artifacts are represented using `types.Part` objects:

```python
from google.genai.types import Part, Blob

img_part = Part(
    inline_data=Blob(
        mime_type="image/png",
        data=b'\x89PNG...'  # binary PNG data
    )
)
```


## 🤔 Why Use Artifacts? <a id="why-use-artifacts"></a>

| Use Case | Why Artifacts Work |
|----------|--------------------|
| File upload/download | Easy to persist user files |
| Report/image generation | Save and reuse across steps |
| Audio or PDF generation | Supports rich formats |
| Caching | Store previously computed binary blobs |
| Cross-session reuse | Use `user:` prefix for long-lived files |

Artifacts solve a key gap — handling large, versioned **non-textual** or **binary** content.


## ⚙️ Core Concepts <a id="core-concepts"></a>

### 🔧 ArtifactService

The **backend** for storing, retrieving, and listing artifacts.

```python
from google.adk.artifacts import InMemoryArtifactService

artifact_service = InMemoryArtifactService()
```

Each service implements:

- `save_artifact`
- `load_artifact`
- `list_artifacts`
- `delete_artifact` (backend-only)
- `list_versions`

### 🧱 Artifact Data

```python
from google.genai.types import Part
img = Part.from_bytes(data=b'\x89PNG...', mime_type="image/png")
```

### 🏷️ Filename & Namespacing

| Scope | Example | Shared across |
|-------|---------|----------------|
| Session | `"summary.pdf"` | Same session |
| User | `"user:avatar.jpg"` | All sessions by user |


## 🔁 Interacting with Artifacts <a id="interacting-with-artifacts"></a>

You interact using `CallbackContext` or `ToolContext`.

### ✅ Saving Artifacts

```python
version = await context.save_artifact("report.pdf", my_part)
```

### 📤 Loading Artifacts

```python
part = await context.load_artifact("report.pdf")
data = part.inline_data.data
```

### 📋 Listing Artifacts

```python
files = await context.list_artifacts()
```

> Requires `artifact_service` set on `Runner(...)`


## 🛠️ ArtifactService Implementations <a id="artifactservice-implementations"></a>

### 🧪 InMemoryArtifactService

- Fast and stateless
- Lost after restart
- Great for local testing

```python
from google.adk.artifacts import InMemoryArtifactService
artifact_service = InMemoryArtifactService()
```

### ☁️ GcsArtifactService

- Persistent via Google Cloud
- Supports IAM auth and multiple users

```python
from google.adk.artifacts import GcsArtifactService
gcs_service = GcsArtifactService(bucket_name="my-bucket")
```


## ✅ Best Practices <a id="best-practices"></a>

- Use `mime_type` always (e.g. `"application/pdf"`)
- Keep artifact sizes moderate
- Clean up if using `GcsArtifactService`
- Use `user:` prefix for cross-session access
- Treat `save_artifact` as append-only (versioned)

**Avoid storing artifacts inside `session.state`!** Artifacts are versioned binary blobs, not config flags.
